# 2017-04-09
----
Reminders : I'm trying to write a pythn-on script that run as background (daemon) on the Pi, that allow to interact with few command lines (API, maybe using zmq and/or socket) to do 3 tasks :
- Ask the camera if it's ready to record
- Allow the camera to answer positively or negatively
- Ask the camera to cut down the connection  


- [ ] Create a daemon

Like last week, I have connection problems to the Pi. But this time, restarting it didn't solved the problem. After talking with L. Perrinet, we didn't found where it come from.  
I'll work on the daemon/API script until further notice.  
More informations about zmq : http://zeromq.org/, http://api.zeromq.org/ and http://zguide.zeromq.org/py:all  
Going further with the interactive python scripts : https://docs.python.org/2/library/thread.html#module-thread and https://docs.python.org/2/library/threading.html  

The daemon script is under construction : pi_interaction.ipynb

The connection to the Pi works with a new IP : __10.164.7.213__

    ssh pi@10.164.7.213

To learn how zmq is working, I'm reading documentation : http://zguide.zeromq.org/py:all  
The zmq python library isn't installed on the pi. Problem : When trying to install it through

    sudo aptitude install python3-zmq

output :

    E: Could not get lock /var/lib/dpkg/lock - open (11: Resource temporarily unavailable)
    E: Unable to lock the administration directory (/var/lib/dpkg/), is another process using it?
    W: Could not lock the cache file; this usually means that dpkg or another apt tool is already installing packages.
    (...)
    
But no other apt process should be running.

    ps aux | grep apt

output 3 apt process running :

    root       747  0.0  0.3   6184  2956 pts/0    S+   14:26   0:00 sudo apt-get upgrade
    root       751  7.3  3.4  33656 30592 pts/0    S+   14:26   3:32 apt-get upgrade
    pi       13293  0.0  0.1   4248  1432 pts/2    S+   15:14   0:00 grep --color=auto apt
    
After verification, they were launched by L. Perrinet. Waiting their end to install zmq.  
zmq python library is succesfully installed on the pi.

The following scripts works on a local machine but doesn't initiate a computer/pi connection :

    '''Server side'''

    import zmq

    context = zmq.Context()
    print('Connecting to client...')
    socket = context.socket(zmq.REQ)
    socket.connect("tcp://localhost:5555")
    print('Client connected')

    for request in range(10):
        print("Sending request %s..." % request)
        socket.send(b"Hello")

        message = socket.recv() #Wait for next request from client
        print("Received reply %s [ %s ]" % (request, message))

    print('No more requests')
  

    '''Client side'''

    import time
    import zmq

    context = zmq.Context()
    socket = context.socket(zmq.REP)
    socket.bind("tcp://*:5555")
    print('Server connected')

    while True:
        message = socket.recv() #Wait for next request from client
        print("Received request: %s" % message)

        time.sleep(2)
        socket.send(b"World") #Send reply back to client
        
More informations about print(b"sting") : http://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal. It allow the string to be converted into bytes, necessary to be usable for other programming language.  

To make these scripts works with a computer/pi connection, we need to modify the socket.connect parameters, from

    socket.connect("tcp://localhost:5555")
    
to 

    socket.connect("tcp://10.164.7.213:5555")
    
Then client (pi) output :

    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    
While server (computer) output :

    Connecting to client...
    Client connected
    Sending request 0...
    Received reply 0 [ b'World' ]
    Sending request 1...
    Received reply 1 [ b'World' ]
    Sending request 2...
    Received reply 2 [ b'World' ]
    Sending request 3...
    Received reply 3 [ b'World' ]
    Sending request 4...
    Received reply 4 [ b'World' ]
    Sending request 5...
    Received reply 5 [ b'World' ]
    Sending request 6...
    Received reply 6 [ b'World' ]
    Sending request 7...
    Received reply 7 [ b'World' ]
    Sending request 8...
    Received reply 8 [ b'World' ]
    Sending request 9...
    Received reply 9 [ b'World' ]
    No more requests
